# Part 4: Image segmentation


In this example we will show segmentation of a Landsat scene.
We show two segmentation modules: [i.segment](https://grass.osgeo.org/grass-stable/manuals/i.segment.html) and the addon [i.superpixels.slic](https://grass.osgeo.org/grass-stable/manuals/addons/i.superpixels.slic.html).
Note that each segmentation algorithm is designed for different purpose, so we can't directly compare them.

First, let's create a new mapset *segmentation* in nc_spm_08_grass7 location:

In [ ]:
%%bash
grass -c -e ~/grassdata/nc_spm_08_grass7/segmentation

Initialize GRASS session:

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init(os.path.expanduser("~/grassdata"), "nc_spm_08_grass7", "segmentation")

Install the addon:

In [ ]:
gs.run_command("g.extension", extension="i.superpixels.slic")

### Data preparation
Imagery modules typically work with *imagery groups*. We first list the landsat raster data and then create an imagery group:



In [ ]:
maps = gs.list_grouped(type="raster", pattern="lsat*")["PERMANENT"]
print(maps)
gs.run_command("i.group", group="landsat", subgroup="landsat", input=maps)

Next, we derive NDVI to see some of the effects of segmentation:

In [ ]:
gs.run_command("g.region", raster="lsat7_2002_30")
gs.run_command("i.vi", red="lsat7_2002_30",  nir="lsat7_2002_40", output="ndvi", viname="ndvi")

img = gj.Map()
img.d_rast(map="ndvi")
img.d_legend(raster="ndvi", at=(5, 50, 0, 5))
img.show()

### Segmentation with i.superpixels.slic
Superpixels can be defined as a group of pixels that share common characteristics and are useful in computer vision and image processing.
Here we run i.superpixels.slic and convert the resulting raster to vector for better visualization.

In [ ]:
gs.run_command("i.superpixels.slic", input="landsat", output="superpixels", num_pixels=1500, compactness=0.8)
gs.run_command("r.to.vect", input="superpixels", output="superpixels", type="area")

You can play with *compactness* and *num_pixels* parameters and see how the resulting segmentation changes:

In [ ]:
gs.run_command("v.rast.stats", map="superpixels", raster="ndvi", type="area", method="average", column_prefix="superpixels")
gs.run_command("v.to.rast", input="superpixels", output="superpixels_ndvi", type="area", use="attr", attribute_column="superpixels_average")

In [ ]:
gs.run_command("r.colors", map="superpixels_ndvi", color="ndvi")

In [ ]:
img = gj.Map(width=1200)
img.d_rast(map="superpixels_ndvi")
img.d_vect(map="superpixels", width=1, color="#002306", fill_color="none")
#img.d_legend(raster="ndvi", at=(5, 50, 0, 5))
img.show()

For fun, let's do zonal statistics on the results. We compute the median NDVI value within each segment:

In [ ]:
img.save("i_superpixels_slic_ndvi_1200.png")

In [ ]:

gs.run_command("r.stats.quantile", base="superpixels", cover="ndvi", output="superpixels_ndvi")

img = gj.Map()
img.d_rast(map="superpixels_ndvi")
img.d_legend(raster="superpixels_ndvi", at=(5, 50, 0, 5))
img.show()

### Segmentation with i.segment

Next, we do the same, but with i.segment to see the different behavior. Note that i.segment uses *region growing* algorithm by default, but *mean shift* is also available:

In [ ]:
gs.run_command("i.segment", group="landsat", output="segments", threshold=0.5, minsize=50)
gs.run_command("r.to.vect", input="segments", output="segments", type="area")

In [ ]:
img = gj.Map()
img.d_rast(map="ndvi")
img.d_vect(map="segments", width=1, color="black", fill_color="none")
img.d_legend(raster="ndvi", at=(5, 50, 0, 5))
img.show()

In [ ]:
gs.run_command("r.stats.quantile", base="segments", cover="ndvi", output="segments_ndvi")

img = gj.Map()
img.d_rast(map="segments_ndvi")
img.d_legend(raster="superpixels_ndvi", at=(5, 50, 0, 5))
img.show()